In [1]:
import pandas as pd
import numpy as np

In [2]:
df_raw: pd.DataFrame = pd.read_csv(
    '../data/processed_data.csv',
    parse_dates=["Time"],
    index_col=[0],
    sep=';'
)

threshold = 1500
df = df_raw[df_raw['Corrente motor'] > threshold]

# Drop all rows with NaN values
df = df.dropna().copy()

In [3]:
def create_vibration_windows(df, feature_column='feature', window_size=30, threshold=1.5):
    # Step 1: Identify indices where the feature column value is greater than the threshold
    end_indices = np.where(df[feature_column] > threshold)[0]

    # Step 2: Create windows of the specified size ending at these indices
    windows = []
    used_indices = set()

    for idx in end_indices:
        start_idx = idx - (window_size - 1)  # Calculate the start of the window
        if start_idx < 0:
            continue  # Skip windows that start before the beginning of the DataFrame

        # Check if the window overlaps with any already used indices
        if any(i in used_indices for i in range(start_idx, idx + 1)):
            continue  # Skip overlapping windows

        # Add the window to the list of windows
        windows.append(df.iloc[start_idx:idx + 1])

        # Mark the indices as used
        used_indices.update(range(start_idx, idx + 1))

    return windows

def create_non_vibration_windows(df, critical_windows, feature_column='feature', threshold=1.5):
    # Criar janelas não críticas (exemplo: janelas aleatórias abaixo do threshold)
    non_critical_windows = []
    for _ in range(len(critical_windows)):  # Criar o mesmo número de janelas não críticas
        start_idx = np.random.randint(0, len(df) - 30)  # Escolher um índice aleatório
        window = df.iloc[start_idx:start_idx + 30]
        if window[feature_column].max() <= threshold:  # Garantir que a janela não é crítica
            non_critical_windows.append(window)

    return non_critical_windows

def remove_problematic_columns(df):
    # Lista para armazenar colunas a serem removidas
    columns_to_drop = []

    # Verifica cada coluna no DataFrame
    for col in df.columns:
        # Verifica se a coluna contém tipos de dados não numéricos
        if df[col].dtype == 'object':
            print(f"Coluna '{col}' removida: contém tipo de dados 'object'.")
            columns_to_drop.append(col)
            continue

        # Verifica se a coluna contém valores não numéricos (NaN, infinito, etc.)
        if not np.isfinite(df[col]).all():
            print(f"Coluna '{col}' removida: contém valores não numéricos (NaN ou infinito).")
            columns_to_drop.append(col)

    # Remove as colunas problemáticas
    df_cleaned = df.drop(columns=columns_to_drop)

    return df_cleaned

In [4]:
df = remove_problematic_columns(df)

critical_windows = create_vibration_windows(df)
non_critical_windows = create_non_vibration_windows(df, critical_windows)

# Rotular as janelas
critical_labels = [1] * len(critical_windows)  # 1 para críticas
non_critical_labels = [0] * len(non_critical_windows)  # 0 para não críticas

# Combinar janelas e rótulos
windows = critical_windows + non_critical_windows
labels = critical_labels + non_critical_labels

# Converter janelas para numpy array (formato adequado para modelos)
X = np.array([window.values for window in windows])  # Usar todas as colunas da janela
y = np.array(labels)

Coluna 'Unnamed: 58' removida: contém tipo de dados 'object'.


In [5]:
from sklearn.model_selection import train_test_split

# Dividir os dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [6]:
X_train.shape

(2052, 30, 69)

In [7]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

# Construir o modelo
model = Sequential([
    LSTM(64, input_shape=(X_train.shape[1], X_train.shape[2])),  # (timesteps, n_features)
    Dropout(0.2),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')  # Saída binária (crítica ou não crítica)
])

# Compilar o modelo
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Treinar o modelo
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2)

# Avaliar o modelo
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Acurácia: {accuracy:.2f}")

2025-03-06 16:58:20.851180: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-06 16:58:20.851632: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-06 16:58:20.853673: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-06 16:58:20.858312: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1741280300.865889    3596 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1741280300.86

Epoch 1/50


2025-03-06 16:58:21.887089: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)
/usr/local/lib/python3.9/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


52/52 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.5256 - loss: 0.7108 - val_accuracy: 0.6131 - val_loss: 0.6676
Epoch 2/50
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6169 - loss: 0.6665 - val_accuracy: 0.6131 - val_loss: 0.6642
Epoch 3/50
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6252 - loss: 0.6596 - val_accuracy: 0.6131 - val_loss: 0.6671
Epoch 4/50
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6291 - loss: 0.6602 - val_accuracy: 0.6131 - val_loss: 0.6644
Epoch 5/50
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6420 - loss: 0.6525 - val_accuracy: 0.6131 - val_loss: 0.6612
Epoch 6/50
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6350 - loss: 0.6565 - val_accuracy: 0.6131 - val_loss: 0.6615
Epoch 7/50
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6095 - loss: 0.6618 - val_accuracy: 0.6131 - val_loss: 0.6651
Epoch 8/50
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6412 - loss: 0.6565 - val_accuracy: 0.6131 - val_loss: 0.6633
Epo